sudo service mongod start

TODO:
1. Import the companies.json dataset into a mongodb collection
2. Using pymongo connect to the database and:
    1. From all the companies get first office, number of people and company name
    2. Convert it to GeoJSON format
    3. Insert it back to another mongodb collection
3. In another notebook or python file
    1. Query the newly created collection geospatially using $near operator and for each company calculate the nearest companies with radius R meters. Use R=100, R=1000 and R=10000
    2. For each company sum the results of the nearest companies and save all results in a JSON file containing an array with the following format: 
[ 
    {
        id: XXXX
        name: <company_name>, 
        office_location:<geojson_point_office_location>,
        nearest_workers:{ 
            “r=100”:9999,
            “r=1000”:9999,
            “r=10000”:9999,
        }
    },
    {…},
    {...}
]

4. Using the created json file, add it to tableau and create 3 sheets using geospartial plot.
5. Create a tableau story and present the results you’ve found

In [136]:
import pandas as pd
from pandas.io.json import json_normalize
import pymongo
from pymongo import MongoClient
from pprint import pprint
from geojson import Point
import numpy as np

In [137]:
client = MongoClient('localhost',27017)
db = client.local
collection = db.demofile

Cuento el número de registros para ver cómo es de grande mi  bbdd

In [138]:
registros=collection.count_documents({})
registros

5614

In [139]:
q=collection.find(projection={'name':1,'geopoint':1,'number_of_employees':1,'_id':0})
df=pd.DataFrame(list(q))
df.head()

,geopoint,name,number_of_employees
0,"{'type': 'Point', 'coordinates': [-122.333253,...",Wetpaint,47
1,"{'type': 'Point', 'coordinates': [-122.394523,...",Digg,60
2,"{'type': 'Point', 'coordinates': [-121.904945,...",Zoho,1600
3,"{'type': 'Point', 'coordinates': [-118.393064,...",Geni,18
4,"{'type': 'Point', 'coordinates': [-121.904945,...",AdventNet,600


Hago prueba  con la función de Marc con un una longitud y una latitud concretas para ver qué resultados arroja.

In [140]:
def geoQueryNear(geoJsonField, lat, lng, distance):
      return {geoJsonField: {"$near":{"$geometry": {"type":"Point","coordinates": [lat, lng]},"$maxDistance":distance}}}
    
        
result = collection.find(geoQueryNear('geopoint',-122.333253,47.603122, 100),projection={'name':1,'number_of_employees':1,'geopoint':1,'_id':0})

df2= pd.DataFrame(list(result))


Query the newly created collection geospatially using $near operator and for each company calculate the nearest companies with radius R meters. Use R=100, R=1000 and R=10000

Ahora defino una función para que, en función de la distancia, me devuelva las compañías que están en esa distancia para cada uno de los registros del db original. Compruebo antes que accedo de la manera correcta a la longitud y latitud

In [141]:
print(df['geopoint'][0])
print(df['geopoint'][2555]['coordinates'][0])
print(df['geopoint'][2555]['coordinates'][1])

{'type': 'Point', 'coordinates': [-122.333253, 47.603122]}
-73.1762626
40.8887993


DEfino la función con el parámetro distancia y creo una lista vacía para que añada por cada long y lat la suma del número de empleados de las empresas que están en ese radio

In [142]:
def radio(n):
    suma=[]
    for i in range(registros):
        lat= df['geopoint'][i]['coordinates'][0]
        long= df['geopoint'][i]['coordinates'][1]
        result= collection.find(geoQueryNear('geopoint',lat,long, n),projection={'number_of_employees':1,'_id':0})
        result=pd.DataFrame(list(result))    
        suma_emp= result['number_of_employees'].sum()
        suma.append(suma_emp)
        
    return suma
radio(100)
radio(1000)
radio(10000)

[4904,
 22265,
 4123,
 27719,
 4123,
 3558,
 22272,
 172940,
 173180,
 22265,
 394239,
 72947,
 415472,
 22272,
 22272,
 275322,
 944,
 3394,
 22263,
 40724,
 2892,
 172672,
 4920,
 330769,
 107440,
 262906,
 1535,
 1942,
 28949,
 172672,
 33,
 172673,
 297918,
 238976,
 67,
 22265,
 107,
 382669,
 22676,
 107,
 22272,
 344606,
 1278,
 53,
 22272,
 125215,
 670761,
 22272,
 12775,
 12775,
 2,
 19614,
 360953,
 126098,
 378072,
 4888,
 3174,
 22265,
 19311,
 110841,
 22272,
 22272,
 173185,
 22272,
 382669,
 509,
 4892,
 110247,
 22263,
 297918,
 22275,
 5020,
 76832,
 5277,
 4888,
 284,
 27183,
 74367,
 11025,
 107184,
 22265,
 4904,
 279,
 24,
 131249,
 22265,
 22265,
 1215,
 16254,
 90,
 40551,
 22480,
 126118,
 89730,
 375,
 22274,
 28202,
 319268,
 22272,
 9027,
 22265,
 509,
 5,
 298066,
 77049,
 403,
 82734,
 4121,
 22272,
 645903,
 22272,
 22265,
 2,
 5080,
 107014,
 176101,
 69076,
 55,
 11944,
 569,
 52,
 11881,
 22272,
 12775,
 22265,
 22272,
 191166,
 2892,
 22265,
 16573,
 

Ahora añado cada lista al dataframe inicial

In [143]:
df['suma_100']= radio(100)
df['suma_1000']=radio(1000)
df['suma_10000']=radio(10000)
df.head()


,geopoint,name,number_of_employees,suma_100,suma_1000,suma_10000
0,"{'type': 'Point', 'coordinates': [-122.333253,...",Wetpaint,47,352,1722,4904
1,"{'type': 'Point', 'coordinates': [-122.394523,...",Digg,60,60,763,22265
2,"{'type': 'Point', 'coordinates': [-121.904945,...",Zoho,1600,2200,2732,4123
3,"{'type': 'Point', 'coordinates': [-118.393064,...",Geni,18,18,18,27719
4,"{'type': 'Point', 'coordinates': [-121.904945,...",AdventNet,600,2200,2732,4123


In [168]:
result = json_normalize(df['geopoint'])
result=result.drop(['type'],axis=1)
result.head()
total= pd.concat([df, result], axis=1)
total=total.drop(['geopoint'],axis=1)
tota